In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction

Inspired by this [notebook](https://www.kaggle.com/pintowar/otimiza-o-de-carteira-de-criptomoedas) I'll create a portfolio with 4 cryptocurrencies using Markowitz Model. Each semester portfolio will be rebalanced semiannually. 

There will be three portfolios, one with minimum volatility,one with highest sharpe index and other equally balanceded (each crypto represents 25% of the portfolio).

Portfolios will be generated using a simulation that you can find [Here](http://towardsdatascience.com/python-markowitz-optimization-b5e1623060f5).

This notebook is divided in six parts:
- Data preparation
- Evolution through the years of Cryptocurrencies
- Correlation
- Minimum Volatility Portfolio
- Maximum Sharpe Portfolio
- Equally Balanced Portfolio
- Conclusion

If you find any mistake or have any suggestion, feel free to comment.

**Note:** To calculate Sharpe Index I won't use risk free asset.I'll calculate the ratio between return and volatility.

## Data Preparation

In [ ]:
# Import libraries
from statistics import mean
import statistics
import matplotlib.pyplot as plt
import math

# upload files
eth = pd.read_csv("../input/crypto-currencies-historical-data/ETH-USD.csv")
eth = eth.round(2)

xrp = pd.read_csv("../input/crypto-currencies-historical-data/XRP-USD.csv")
xrp = xrp.round(2)

btc = pd.read_csv("../input/crypto-currencies-historical-data/BTC-USD.csv")
btc = btc.round(2)

ltc = pd.read_csv("../input/crypto-currencies-historical-data/LTC-USD.csv")
ltc = ltc.round(2)

# Create a dataframe with only close price
eth_cls = eth['Close'].tolist()
xrp_cls = xrp['Close'].tolist()
btc_cls = btc['Close'].tolist()
ltc_cls = ltc['Close'].tolist()

lista = [eth_cls,xrp_cls,btc_cls,ltc_cls]

resumo = pd.DataFrame(lista)
resumo = resumo.transpose()

resumo.columns = ['Eth','Xrp','Btc','Ltc']
resumo['Date'] = eth['Date'].tolist()

# Let's add a column for semester
resumo = resumo[['Eth','Xrp','Btc','Ltc','Date']]
resumo['Date'] = pd.to_datetime(resumo['Date'])
resumo['Sem']= resumo.Date.dt.year.astype(str) + '-S'+ np.where(resumo.Date.dt.quarter.gt(2),2,1).astype(str)

# Create a function to split dataframe by semester
def listar_sem(dataframe):
    new_df = dataframe.copy()  # Copia o dataframe input
    sems = new_df['Sem'].unique() # Vai criar uma array com cada semestre que aparece na coluna
    sems = sems.tolist()    # Transforma a array em uma lista
    
    # Filtrar dataframes
    dbs = []
    for sem in sems:  # Vai pegar os semestres na lista e usar como critério para filtrar os dataframes
        db = new_df.loc[new_df['Sem'] == sem]
        dbs.append(db)
            
    return dbs

# Create dataframe only with prices
prices = resumo[['Eth','Xrp','Btc','Ltc','Date','Sem']]
preços = listar_sem(prices)

# Create a list with all semesters
semesters = resumo['Sem'].unique()

# Create columns for daily returns
var = resumo.copy()
var['Var_Eth'] = (var['Eth']/var['Eth'].shift(1))-1
var['Var_Xrp'] = (var['Xrp']/var['Xrp'].shift(1))-1
var['Var_Btc'] = (var['Btc']/var['Btc'].shift(1))-1
var['Var_Ltc'] = (var['Ltc']/var['Ltc'].shift(1))-1

# Keep only daily returns
var = var[['Var_Eth','Var_Xrp','Var_Btc','Var_Ltc','Date','Sem']]

# Drop first row
var = var.iloc[1:]

# Split by semester
var_sem = listar_sem(var)

# For each semester keep only daily returns
verif = []
for i in range(len(var_sem)):
    df = var_sem[i]
    df = df[['Var_Eth','Var_Xrp','Var_Btc','Var_Ltc']]
    verif.append(df)

## Evolution through the years

Let's check how crypto coins evolved through time. To this I'll normalize prices by making the first price equals to 100. The reason for that is because I'm only interested in price variation not in each coin value.

In [ ]:
# Create function to normalize
def normalize(lista):
    # Input a list of dataframes

    lista_final = []
    
    num = len(lista)
    
    for i in range(num):
        df = lista[i]
        df['Var'] = (df['Close']/df['Close'].shift(1))-1
        df_var = df['Var'].tolist()
        df_norm = [100]
        
        tam = len(df_var)-1
        valor = 0

        for i in range(tam):
            valor = df_norm[i]*(1+df_var[i+1])
            df_norm.append(valor)
            
        lista_final.append(df_norm)
        
    tabela = pd.DataFrame(lista_final)
    tabela = tabela.transpose()
    
    return tabela

# Create a list of dataframe for each crypto
lista_df = [eth,xrp,btc,ltc]

df = normalize(lista_df)

df = df.round(2)

df.columns = ['Eth','Xrp','Btc','Ltc']

df['Date'] = eth['Date'].tolist()

df = df.set_index('Date')

df.plot(figsize=(14,8))

# Correlation

In [ ]:

final = []

for i in range(len(preços)):
    parcial = preços[i]
    parcial_corr = parcial.corr(method='pearson') # Matriz de correlação
    
    lista = []
    
    for n in range(3):       # Coluna
        for j in range(1,4): # Linha
            
            #lista = []
            
            if n < j:
                valor = parcial_corr.iloc[j][n]
                lista.append(valor)
            
            else:
                pass
        
    final.append(lista)
    
eth_xrp = []
eth_btc = []
eth_ltc = []
xrp_btc = []
xrp_ltc = []
btc_ltc = []

for i in range(len(final)):
    
    eth_xrp.append(final[i][0])
    eth_btc.append(final[i][1])
    eth_ltc.append(final[i][2])
    xrp_btc.append(final[i][3])
    xrp_ltc.append(final[i][4])
    btc_ltc.append(final[i][5])
    
grupo = [eth_xrp,eth_btc,eth_ltc,xrp_btc,xrp_ltc,btc_ltc]

pairs = ['eth_xrp','eth_btc','eth_ltc','xrp_btc','xrp_ltc','btc_ltc']
    
minimum = []
quantile1 = []
quantile2 = []
quantile3 = []
maximum = []

for i in range(len(grupo)):
    
    alvo = grupo[i]
    
    minimum.append(min(alvo))
    quantile1.append(np.quantile(alvo, .25))
    quantile2.append(np.quantile(alvo, .5))
    quantile3.append(np.quantile(alvo, .75))
    maximum.append(max(alvo))
    
# Criar Dataframe
resumo_corr = pd.DataFrame(list(zip(pairs, minimum, quantile1, quantile2, quantile3,
                                    maximum)), 
               columns =['Pair', 'Minimum','1º Quantile','Median','3º Quantile',
                         'Maximum'])

resumo_corr.head(6)

In [ ]:
x = {'eth_xrp':eth_xrp,'eth_btc':eth_btc,'eth_ltc':eth_ltc,'xrp_btc':xrp_btc,
     'xrp_ltc':xrp_ltc,'btc_ltc':btc_ltc}
summary_corr = pd.DataFrame(x)
summary_corr['Semester'] = semesters
summary_corr = summary_corr.round(4)
summary_corr.head(10)

summary_corr.plot.bar(x='Semester', y='eth_xrp', style='-', figsize=(6,3))
summary_corr.plot.bar(x='Semester', y='eth_btc', style='-', figsize=(6,3))
summary_corr.plot.bar(x='Semester', y='eth_ltc', style='-', figsize=(6,3))
summary_corr.plot.bar(x='Semester', y='xrp_btc', style='-', figsize=(6,3))
summary_corr.plot.bar(x='Semester', y='xrp_ltc', style='-', figsize=(6,3))
summary_corr.plot.bar(x='Semester', y='btc_ltc', style='-', figsize=(6,3))

## Minimum Volatility Portfolio

In [ ]:
# Generate portfolios
# Create a function to return weights for minimum volatility portfolio
# Function receives a list of dataframes and the number of simulations to be done

def port_vol(lista,num):
    
    pesos_finais = []
    
    for i in range(len(lista)):
        
        verif = lista[i]
        
        np.random.seed(42)
        num_ports = num
        all_weights = np.zeros((num_ports, len(verif.columns)))
        ret_arr = np.zeros(num_ports)
        vol_arr = np.zeros(num_ports)
        sharpe_arr = np.zeros(num_ports)

        for x in range(num_ports):
            # Weights
            weights = np.array(np.random.random(4))
            weights = weights/np.sum(weights)
    
            # Save weights
            all_weights[x,:] = weights
    
            # Expected return
            ret_arr[x] = np.sum( (verif.mean() * weights * 252))
    
            # Expected volatility
            vol_arr[x] = np.sqrt(np.dot(weights.T, np.dot(verif.cov()*252, weights)))
    
            # Sharpe Ratio
            sharpe_arr[x] = ret_arr[x]/vol_arr[x]
            
        # Find minimum Volatility
        vol_ind = np.where(vol_arr == np.amin(vol_arr))
        vol_ind = vol_ind[0]
        pesos_vol = all_weights[vol_ind]
        pesos_vol = list(pesos_vol[0])
            
        pesos_finais.append(pesos_vol) 
            
    return pesos_finais

pesos_ativos = port_vol(verif,5000)

In [ ]:
# Split by semester
preços_sem = listar_sem(prices)

quantidades = []

budget = 1000

import math

for i in range(len(preços_sem)):
    
    df_preços = preços_sem[i]
    lista_pesos = pesos_ativos[i]
    
    peso1 = lista_pesos[0]
    peso2 = lista_pesos[1]
    peso3 = lista_pesos[2]
    peso4 = lista_pesos[3]
    
    limite1 = budget*peso1
    limite2 = budget*peso2
    limite3 = budget*peso3
    limite4 = budget*peso4
    
    valor1 = df_preços.iloc[0][0]
    valor2 = df_preços.iloc[0][1]
    valor3 = df_preços.iloc[0][2]
    valor4 = df_preços.iloc[0][3]
    
    qtde1 = (math.floor((limite1/valor1)*100))/100 
    qtde2 = (math.floor((limite2/valor2)*100))/100    
    qtde3 = (math.floor((limite3/valor3)*100))/100
    qtde4 = (math.floor((limite4/valor4)*100))/100
    
    caixa = budget - qtde1*valor1 - qtde2*valor2 - qtde3*valor3 - qtde4*valor4
    
    valor_final1 = df_preços.iloc[-1][0]
    valor_final2 = df_preços.iloc[-1][1]
    valor_final3 = df_preços.iloc[-1][2]
    valor_final4 = df_preços.iloc[-1][3]
    
    budget = qtde1 * valor_final1 + qtde2 * valor_final2 + qtde3 * valor_final3 + qtde4 * valor_final4 + caixa
    
    valores = [qtde1,qtde2,qtde3,qtde4,caixa]
    
    quantidades.append(valores)
    
# Create dataframe
referencia = pd.DataFrame(quantidades)
referencia.columns = ['Qtde ETH','Qtde XRP','Qtde BTC','Qtde LTC','Caixa']
referencia['Sem'] = semesters

df_final = pd.merge(prices, referencia)
df_final['Valor'] = (df_final['Eth']*df_final['Qtde ETH'] + df_final['Xrp']*df_final['Qtde XRP']
                     + df_final['Btc']*df_final['Qtde BTC'] + df_final['Ltc']*df_final['Qtde LTC']
                     + df_final['Caixa'])

# Absolute Return
absolute_return = (df_final.iloc[-1][11]/df_final.iloc[0][11])-1

retorno_eth = (df_final.iloc[-1][0]/df_final.iloc[0][0])-1
retorno_xrp = (df_final.iloc[-1][1]/df_final.iloc[0][1])-1
retorno_btc = (df_final.iloc[-1][2]/df_final.iloc[0][2])-1
retorno_ltc = (df_final.iloc[-1][3]/df_final.iloc[0][3])-1

print('Min. Vol Portfolio had an absolute return of ' + 
      str(round(absolute_return,2)) + '%')

In [ ]:
df_final.plot(x='Date', y='Valor', style='-', figsize=(14,8))

## Maximum Sharpe Portfolio

In [ ]:
# Generate portfolios
# Create a function to return weights for maximum sharpe portfolio
# Function receives a list of dataframes and the number of simulations to be done

def port_sharpe(lista,num):
    
    pesos_finais = []
    
    for i in range(len(lista)):
        
        verif = lista[i]
        
        np.random.seed(42)
        num_ports = num
        all_weights = np.zeros((num_ports, len(verif.columns)))
        ret_arr = np.zeros(num_ports)
        vol_arr = np.zeros(num_ports)
        sharpe_arr = np.zeros(num_ports)

        for x in range(num_ports):
            # Weights
            weights = np.array(np.random.random(4))
            weights = weights/np.sum(weights)
    
            # Save weights
            all_weights[x,:] = weights
    
            # Expected return
            ret_arr[x] = np.sum( (verif.mean() * weights * 252))
    
            # Expected volatility
            vol_arr[x] = np.sqrt(np.dot(weights.T, np.dot(verif.cov()*252, weights)))
    
            # Sharpe Ratio
            sharpe_arr[x] = ret_arr[x]/vol_arr[x]
            
        # Find Max sharpe
        sharpe_ind = np.where(sharpe_arr == np.amax(sharpe_arr))
        sharpe_ind = sharpe_ind[0]
        pesos_sharpe = all_weights[sharpe_ind]
        pesos_sharpe = list(pesos_sharpe[0])
            
        pesos_finais.append(pesos_sharpe) 
            
    return pesos_finais

pesos_ativos = port_sharpe(verif,5000)

In [ ]:

quantidades2 = []

budget = 1000

import math

for i in range(len(preços_sem)):
    
    df_preços = preços_sem[i]
    lista_pesos = pesos_ativos[i]
    
    peso1 = lista_pesos[0]
    peso2 = lista_pesos[1]
    peso3 = lista_pesos[2]
    peso4 = lista_pesos[3]
    
    limite1 = budget*peso1
    limite2 = budget*peso2
    limite3 = budget*peso3
    limite4 = budget*peso4
    
    valor1 = df_preços.iloc[0][0]
    valor2 = df_preços.iloc[0][1]
    valor3 = df_preços.iloc[0][2]
    valor4 = df_preços.iloc[0][3]
    
    qtde1 = (math.floor((limite1/valor1)*100))/100 
    qtde2 = (math.floor((limite2/valor2)*100))/100    
    qtde3 = (math.floor((limite3/valor3)*100))/100
    qtde4 = (math.floor((limite4/valor4)*100))/100
    
    caixa = budget - qtde1*valor1 - qtde2*valor2 - qtde3*valor3 - qtde4*valor4
    
    valor_final1 = df_preços.iloc[-1][0]
    valor_final2 = df_preços.iloc[-1][1]
    valor_final3 = df_preços.iloc[-1][2]
    valor_final4 = df_preços.iloc[-1][3]
    
    budget = qtde1 * valor_final1 + qtde2 * valor_final2 + qtde3 * valor_final3 + qtde4 * valor_final4 + caixa
    
    valores = [qtde1,qtde2,qtde3,qtde4,caixa]
    
    quantidades2.append(valores)
    
# Create dataframe
referencia2 = pd.DataFrame(quantidades2)
referencia2.columns = ['Qtde ETH','Qtde XRP','Qtde BTC','Qtde LTC','Caixa']
referencia2['Sem'] = semesters

df_final2 = pd.merge(prices, referencia2)
df_final2['Valor'] = (df_final2['Eth']*df_final2['Qtde ETH'] + df_final2['Xrp']*df_final2['Qtde XRP']
                     + df_final2['Btc']*df_final2['Qtde BTC'] + df_final2['Ltc']*df_final2['Qtde LTC']
                     + df_final2['Caixa'])

# Absolute Return
absolute_return = (df_final2.iloc[-1][11]/df_final2.iloc[0][11])-1

print('Max. Sharpe Portfolio had an absolute return of ' + 
      str(round(absolute_return,2)) + '%')

In [ ]:
df_final2.plot(x='Date', y='Valor', style='-', figsize=(14,8))

## Equally Balanced Portfolio

In [ ]:
quantidades3 = []

budget = 1000

import math

for i in range(len(preços_sem)):
    
    df_preços = preços_sem[i]
    
    limite1 = budget*0.25
    limite2 = budget*0.25
    limite3 = budget*0.25
    limite4 = budget*0.25
    
    valor1 = df_preços.iloc[0][0]
    valor2 = df_preços.iloc[0][1]
    valor3 = df_preços.iloc[0][2]
    valor4 = df_preços.iloc[0][3]
    
    qtde1 = (math.floor((limite1/valor1)*100))/100 
    qtde2 = (math.floor((limite2/valor2)*100))/100    
    qtde3 = (math.floor((limite3/valor3)*100))/100
    qtde4 = (math.floor((limite4/valor4)*100))/100
    
    caixa = budget - qtde1*valor1 - qtde2*valor2 - qtde3*valor3 - qtde4*valor4
    
    valor_final1 = df_preços.iloc[-1][0]
    valor_final2 = df_preços.iloc[-1][1]
    valor_final3 = df_preços.iloc[-1][2]
    valor_final4 = df_preços.iloc[-1][3]
    
    budget = qtde1 * valor_final1 + qtde2 * valor_final2 + qtde3 * valor_final3 + qtde4 * valor_final4 + caixa
    
    valores = [qtde1,qtde2,qtde3,qtde4,caixa]
    
    quantidades3.append(valores)

# Create Dataframe
referencia3 = pd.DataFrame(quantidades3)
referencia3.columns = ['Qtde ETH','Qtde XRP','Qtde BTC','Qtde LTC','Caixa']
referencia3['Sem'] = semesters

df_final3 = pd.merge(prices, referencia3)
df_final3['Valor'] = (df_final3['Eth']*df_final3['Qtde ETH'] + df_final3['Xrp']*df_final3['Qtde XRP']
                     + df_final3['Btc']*df_final3['Qtde BTC'] + df_final3['Ltc']*df_final3['Qtde LTC']
                     + df_final3['Caixa'])

# Absolute Return
absolute_return = (df_final3.iloc[-1][11]/df_final3.iloc[0][11])-1

print('Equally balanced Portfolio had an absolute return of ' + 
      str(round(absolute_return,2)) + '%')

In [ ]:
df_final3.plot(x='Date', y='Valor', style='-', figsize=(14,8))

# Conclusion

## Evolution Over Time

In [ ]:
# Create a chart with evolution of each asset
# Transform price columns into lists
port1_lista = df_final['Valor'].tolist()
port2_lista = df_final2['Valor'].tolist()
port3_lista = df_final3['Valor'].tolist()
eth_lista = df_final['Eth'].tolist()
xrp_lista = df_final['Xrp'].tolist()
btc_lista = df_final['Btc'].tolist()
ltc_lista = df_final['Ltc'].tolist()

# Take daily returns
close = [port1_lista,port2_lista,port3_lista,eth_lista,xrp_lista,btc_lista,ltc_lista]

daily_ret = []
for j in range(len(close)):
    parcial = []
    df = close[j]
    for i in range(len(df)-1):
        ret = (df[i+1]/df[i])-1
        parcial.append(ret)
    daily_ret.append(parcial)

# Create a dataframe with daily returns lists
lista_final = []

for i in range(len(daily_ret)):
    
    df_norm = [100]
    
    lista = daily_ret[i]
        
    tam = len(lista)

    for i in range(tam):
        valor = df_norm[i]*(1+lista[i])
        df_norm.append(valor)
            
    lista_final.append(df_norm)
        
    tabela = pd.DataFrame(lista_final)
    tabela = tabela.transpose()

tabela.columns = ['Port Min Vol','Port Max Sharpe','Port Bal','Eth','Xrp','Btc','Ltc']
    
tabela['Date'] = eth['Date'].tolist()

tabela = tabela.set_index('Date')

tabela.plot(figsize=(14,8))

## Sumary

In [ ]:
# Retorno Total de cada Ativo
port_min_vol = (df_final.iloc[-1][11]/df_final.iloc[0][11])-1
port_max_sharpe = (df_final2.iloc[-1][11]/df_final2.iloc[0][11])-1
port_eq_bal = (df_final3.iloc[-1][11]/df_final3.iloc[0][11])-1
retorno_eth = (df_final.iloc[-1][0]/df_final.iloc[0][0])-1
retorno_xrp = (df_final.iloc[-1][1]/df_final.iloc[0][1])-1
retorno_btc = (df_final.iloc[-1][2]/df_final.iloc[0][2])-1
retorno_ltc = (df_final.iloc[-1][3]/df_final.iloc[0][3])-1

retorno = [port_min_vol,port_max_sharpe,port_eq_bal,retorno_eth,retorno_xrp,retorno_btc,retorno_ltc]

# Annual Return
a_port_min_vol = ((1+port_min_vol)**(1/5))-1
a_port_max_sharpe = ((1+port_max_sharpe)**(1/5))-1
a_port_eq_bal = ((1+port_eq_bal)**(1/5))-1
a_retorno_eth = ((1+retorno_eth)**(1/5))-1
a_retorno_xrp = ((1+retorno_xrp)**(1/5))-1
a_retorno_btc = ((1+retorno_btc)**(1/5))-1
a_retorno_ltc = ((1+retorno_ltc)**(1/5))-1

a_retorno = [a_port_min_vol,a_port_max_sharpe,a_port_eq_bal,a_retorno_eth,a_retorno_xrp,a_retorno_btc,a_retorno_ltc]

# Annual Volatility
# Take Volatility for each asset -- Variance of daily returns --
vols = []
for i in range(len(daily_ret)):
    vol = np.var(daily_ret[i])
    vol = vol*(252**(1/2))
    vols.append(vol)

# Construct Dataframe
ativo = ['Port Min Vol','Port Max Sharpe','Port Equal Dist.','ETH','XRP','BTC','LTC']

d = {'Ativo':ativo,'Retorno':retorno, 'Retorno Anual':a_retorno,
     'Volatilidade':vols}
summary_df = pd.DataFrame(d)
summary_df['Sharpe'] = summary_df['Retorno Anual']/summary_df['Volatilidade']
summary_df.head(7)

In [ ]:
tabela.plot(y=['Port Min Vol','Port Max Sharpe','Port Bal'], kind="line", figsize=(12,6))

This notebook was inspired by this [one](https://www.kaggle.com/pintowar/otimiza-o-de-carteira-de-criptomoedas) and the code is an adaptation from this [site](https://towardsdatascience.com/python-markowitz-optimization-b5e1623060f5).

There might be some mistake in my code. If you find any error or have any suggestion please let me know.